# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [1]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []

    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)

    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)

    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right

    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys()

    def is_consistent(self, assignment):
        for var, value in assignment.items():
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [2]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [3]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [4]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [5]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [6]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [7]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [8]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [9]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0

    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None

    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]

    def solve(self):
        self.counter = 0
        return self.recursive_backtracking({})[0]

    def recursive_backtracking(self, assignment):
        if assignment and self.csp.is_complete(assignment):
            return (assignment, False)

        variable = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(variable, assignment):
            assignment[variable] = value
            self.counter += 1
            if self.csp.is_consistent(assignment):
                result, failure = self.recursive_backtracking(assignment)
                if not failure:
                    return (result, False)
                assignment.popitem()
        return (assignment, True)

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [10]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'B', 'NT': 'G', 'SA': 'R', 'Q': 'B', 'NSW': 'G', 'V': 'B', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [11]:
import queue
import copy

In [12]:
class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0

    def MRV_heuristic(self, assignment, remaining_values):
        proposition = []
        for var in self.csp.domains.keys():
            if var not in assignment:
                if not proposition:
                    proposition.append((len(remaining_values[var]), var))
                elif proposition:
                    if len(remaining_values[var]) < proposition[0][0]:
                        proposition = [(len(remaining_values[var]), var)]
                    elif len(remaining_values[var]) == proposition[0][0]:
                        proposition.append((len(remaining_values[var]), var))
        return [x[1] for x in proposition]

    def degree_heuristic(self, proposition, remaining_connections):
        variables = []
        for var in proposition:
            fitness = 0
            for x1, _, x2 in remaining_connections:
                if var == x1 or var == x2:
                    fitness += 1
            variables.append((fitness, var))
        return max(variables, key = lambda x: x[0])[1]

    def select_unassigned_variable(self, assignment, remaining_values, remaining_connections):
        proposition = self.MRV_heuristic(assignment, remaining_values)
        if len(proposition) == 1:
            return proposition[0]
        return self.degree_heuristic(proposition, remaining_connections)

    def order_domain_values(self, variable, remaining_values, remaining_connections):
        priority_queue = queue.PriorityQueue()
        for value in remaining_values[variable]:
            cost = 0
            for var1, x, var2 in remaining_connections:
                if variable == var1:
                    if value in remaining_values[var2]:
                        cost += 1
                if variable == var2:
                    if value in remaining_values[var1]:
                        cost += 1
            priority_queue.put((cost, value))
        return tuple(x[1] for x in priority_queue.queue)

    def reduce_remaining_values(self, remaining_values, variable, value):
        for var1, op, var2 in self.csp.binary:
            if var1 == variable:
                x = list(remaining_values[var2])
                for element in remaining_values[var2]:
                    if not self.csp.verify(value, op, element):
                        x.remove(element)
                remaining_values[var2] = x
            if var2 == variable:
                x = list(remaining_values[var1])
                for element in remaining_values[var1]:
                    if not self.csp.verify(element, op, value):
                        x.remove(element)
                remaining_values[var1] = x
        return remaining_values

    def reduce_remaining_connections(self, remaining_connections, variable):
        for constraint in self.csp.binary:
            if variable in constraint:
                if constraint in remaining_connections:
                    remaining_connections.remove(constraint)
        return remaining_connections

    def solve(self):
        self.counter = 0
        return self.recursive_backtracking({}, dict(self.csp.domains), list(self.csp.binary))

    def recursive_backtracking(self, assignment, remaining_values, remaining_connections):
        if self.csp.is_complete(assignment):
            return assignment

        variable = self.select_unassigned_variable(assignment, remaining_values, remaining_connections)
        new_remaining_connections = self.reduce_remaining_connections(list(remaining_connections), variable)

        for value in self.order_domain_values(variable, remaining_values, remaining_connections):
            assignment[variable] = value
            self.counter += 1
            if self.csp.is_consistent(assignment):
                new_remaining_values = self.reduce_remaining_values(dict(remaining_values), variable, value)
                new_assignment = self.recursive_backtracking(assignment, new_remaining_values, new_remaining_connections)
                if new_assignment:
                    return new_assignment
                assignment.popitem()
        return None

Lets test your implementation by solving the problem of coloring Australia!

In [13]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'B', 'NT': 'G', 'Q': 'R', 'NSW': 'G', 'WA': 'R', 'V': 'R', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

*The number of considered assignments for `RecursiveBacktracking` is significantly greater than the number of considered assignments for `RecursiveBacktrackingWithHeuristics`. The heuristic version takes variables and values to solve the problem in such an order that it rarely makes any wrong assumptions or guesses.*

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

###PUZZLE

In [14]:
puzzle = '''
_ _ 3 _ 2 _ 6 _ _
9 _ _ 3 _ 5 _ _ 1
_ _ 1 8 _ 6 4 _ _
_ _ 8 1 _ 2 9 _ _
7 _ _ _ _ _ _ _ 8
_ _ 6 7 _ 8 2 _ _
_ _ 2 6 _ 9 5 _ _
8 _ _ 2 _ 3 _ _ 9
_ _ 5 _ 1 _ 3 _ _
'''

In [15]:
puzzle = '''
8 _ _ _ _ _ _ _ _
_ _ 3 6 _ _ _ _ _
_ 7 _ _ 9 _ 2 _ _
_ 5 _ _ _ 7 _ _ _
_ _ _ _ 4 5 7 _ _
_ _ _ 1 _ _ _ 3 _
_ _ 1 _ _ _ _ 6 8
_ _ 8 5 _ _ _ 1 _
_ 9 _ _ _ _ 4 _ _
'''

In [16]:
puzzle = '''
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [17]:
sudoku = CSP()

###VARIABLES

In [18]:
index = 1
for y in range(9):
    for x in range(9):
        if puzzle[index] == '_':
            sudoku.addVariable((y, x), {1, 2, 3, 4, 5, 6, 7, 8, 9})
        else:
            sudoku.addVariable((y, x), {int(puzzle[index])})
        index += 2

###CONSTRAINS

In [19]:
for y in range(9):
    for x in range(9):
        box_y, box_x = (y // 3) * 3, (x // 3) * 3
        for i in range(3):
            for j in range(3):
                if (box_y + i != y or box_x + j != x) and ((y, x), '!=', (box_y + i, box_x + j)) not in sudoku.binary and ((box_y + i, box_x + j), '!=', (y, x)) not in sudoku.binary:
                    sudoku.addBinaryConstraint((y, x), '!=', (box_y + i, box_x + j))
        for i in range(9):
            for j in range(9):
                    if (y != i or x != j) and (i == y or j == x) and ((y, x), '!=', (i, j)) not in sudoku.binary and ((i, j), '!=', (y, x)) not in sudoku.binary:
                        sudoku.addBinaryConstraint((y, x), '!=', (i, j))

###SOLUTION

Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [20]:
def visualize(assignment):
    index = 0
    output = ''
    for i in range(9):
        for j in range(9):
            if index % 9 == 0:
                output += '\n'
            if (i, j) in assignment:
                output += str(assignment[(i, j)]) + ' '
            else:
                output += '_ '
            index += 1
    return output

In [21]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
print("Assignment:\n", visualize(assignment), '\n')

print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment:
 
1 2 3 4 5 6 7 8 9 
4 5 6 7 8 9 1 2 3 
7 8 9 1 2 3 4 5 6 
2 6 5 8 7 1 3 9 4 
3 1 7 2 9 4 8 6 5 
8 9 4 6 3 5 2 7 1 
5 7 1 3 6 2 9 4 8 
6 4 2 9 1 8 5 3 7 
9 3 8 5 4 7 6 1 2  

Is consistent? True
Is complete? True
# considered assignments 81


## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [22]:
class RecursiveBacktrackingWithAC3:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0

    def reverse_operator(self, op):
        if op == '=':
            return '='
        if op == '!=':
            return '!='
        if op == '>':
            return '<'
        if op == '<':
            return '>'
        if op == '<=':
            return '>='
        if op == '>=':
            return '<='

    def MRV_heuristic(self, assignment, domains):
        proposition = []
        for var in self.csp.domains.keys():
            if var not in assignment:
                if not proposition:
                    proposition.append((len(domains[var]), var))
                elif proposition:
                    if len(domains[var]) < proposition[0][0]:
                        proposition = [(len(domains[var]), var)]
                    elif len(domains[var]) == proposition[0][0]:
                        proposition.append((len(domains[var]), var))
        return [x[1] for x in proposition]

    def degree_heuristic(self, proposition, binary_constrains):
        variables = []
        for var in proposition:
            fitness = 0
            for x1, _, x2 in binary_constrains:
                if var == x1 or var == x2:
                    fitness += 1
            variables.append((fitness, var))
        return max(variables, key = lambda x: x[0])[1]

    def select_unassigned_variable(self, assignment, domains, binary_constrains):
        proposition = self.MRV_heuristic(assignment, domains)
        if len(proposition) == 1:
            return proposition[0]
        return self.degree_heuristic(proposition, binary_constrains)

    def order_domain_values(self, variable, domains):
        return domains[variable]

    def reduce_binary_constrains(self, assignment, binary_constrains, variable):
        for left, op, right in binary_constrains:
            if left in assignment and right in assignment:
                binary_constrains.remove((left, op, right))
        return binary_constrains

    def AC3(self, domains, binary_constrains):
        arcs = list(binary_constrains)
        length = len(arcs)
        for left, op, right in binary_constrains:
            arcs.append((right, self.reverse_operator(op), left))
        agenda = list(arcs)

        while agenda:
            left1, op1, right1 = agenda.pop(0)
            if self.remove_inconsistant_domains(left1, op1, right1, domains):
                for left2, op2, right2 in arcs:
                    if left1 == right2 and (left2, op2, right2) not in agenda:
                        agenda.append((left2, op2, right2))
        return domains

    def remove_inconsistant_domains(self, left, op, right, domains):
        removed = False
        to_be_removed = []
        for value1 in domains[left]:
            meh = True
            for value2 in domains[right]:
                if self.csp.verify(value1, op, value2):
                    meh = False
                    break
            if meh:
                to_be_removed.append(value1)
                removed = True
        for value in to_be_removed:
            domains[left].remove(value)
        return removed

    def solve(self):
        self.counter = 0
        return self.recursive_backtracking({}, dict(self.csp.domains), list(self.csp.binary))

    def recursive_backtracking(self, assignment, domains, binary_constrains):
        if self.csp.is_complete(assignment):
            return assignment

        domains = self.AC3(domains, binary_constrains)
        variable = self.select_unassigned_variable(assignment, domains, binary_constrains)
        new_binary_constrains = self.reduce_binary_constrains(assignment, list(binary_constrains), variable)

        for value in self.order_domain_values(variable, domains):
            assignment[variable] = value
            self.counter += 1
            if self.csp.is_consistent(assignment):
                new_domains = copy.deepcopy(domains)
                new_domains[variable] = {value}
                new_assignment = self.recursive_backtracking(assignment, new_domains, new_binary_constrains)
                if new_assignment:
                    return new_assignment
                assignment.popitem()
        return None

Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [23]:
solver = RecursiveBacktrackingWithAC3(australia)
assignment = solver.solve()
print("Assignment:", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment: {'SA': 'B', 'NT': 'G', 'Q': 'R', 'NSW': 'G', 'WA': 'R', 'V': 'R', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


In [24]:
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve()
print("Assignment:\n", visualize(assignment), '\n')
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment:
 
1 2 3 4 5 6 8 9 7 
4 5 6 8 9 7 1 2 3 
8 9 7 1 2 3 4 5 6 
2 3 1 9 6 4 5 7 8 
9 6 8 5 7 1 2 3 4 
5 7 4 2 3 8 9 6 1 
3 8 9 6 1 2 7 4 5 
6 1 2 7 4 5 3 8 9 
7 4 5 3 8 9 6 1 2  

Is consistent? True
Is complete? True
# considered assignments 81


------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.